# Exercise 2: Text Processing and Classification using Spark

In [1]:
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
from pyspark.sql.types import MapType, IntegerType, DoubleType
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, HashingTF, IDF, ChiSqSelector
from pyspark.ml.functions import vector_to_array
from operator import add
import re

spark: SparkSession = SparkSession.builder.getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.

In [2]:
with open('../data/stopwords.txt') as f:
    stopwords = set(f.read().splitlines())

In [3]:
K = 75
FILE_PATH = 'hdfs:///user/dic24_shared/amazon-reviews/full/reviews_devset.json' # devset
# FILE_PATH = 'hdfs:///user/dic24_shared/amazon-reviews/full/reviewscombined.json' # full dataset

## Part 1) RDDs

Repeat the steps of Assignment 1, i.e. calculation of chi-square values and output of the sorted top terms per category, as well as the joined dictionary, using RDDs and transformations. Write the output to a file `output_rdd.txt`. Compare the generated `output_rdd.txt` with your generated `output.txt` from Assignment 1 and describe your observations briefly in the submission report (see Part 3).

In [ ]:
rdd = spark.read.json(FILE_PATH).rdd

CPU times: user 2.11 ms, sys: 2.1 ms, total: 4.2 ms
Wall time: 788 ms


In [ ]:
%%time
def preprocessing(row):
    category = row['category']

    # lower text
    # tokenises each line by using whitespaces, tabs, digits, and the characters ()[]{}.!?,;:+=-_"'`~#@&*%€$§\/ as delimiters
    tokens = re.split(r'[^a-zA-Z<>^|]+', row['reviewText'].lower())

    # remove stopwords
    tokens = filter(lambda token: len(token) > 1 and (token not in stopwords), tokens)

    # remove duplicates
    tokens = set(tokens)

    # count all documents in category
    yield (category, None), 1

    # count all documents in category containing token
    for token in tokens:
        yield (category, token), 1


def token_to_key(row):
    (category, token), count = row
    return token, (category, count)


def token_sum(row):
    token, values = row
    counts = {category: count for category, count in values}
    n_t = sum(counts.values())

    for category, count in counts.items():
        yield category, (token, count, n_t)


def chi_squared(row):
    category, values = row

    # dictionary of tokens with their counts and total number of documents
    counts = {token: (count, n_t) for token, count, n_t in values}
    
    # total number of documents in category and dataset
    n_c, n = counts.pop(None)
    
    result = []

    for token, (a, n_t) in counts.items():
        b = n_t - a
        c = n_c - a
        d = n - a - b - c

        chi_squared = n * ((a * d - b * c) ** 2) / ((a + b) * (a + c) * (b + d) * (c + d))
        result.append((chi_squared, token))

    return category, sorted(result, key=lambda x: (-x[0], x[1]))[:K]

CPU times: user 23 µs, sys: 0 ns, total: 23 µs
Wall time: 26.7 µs


In [ ]:
%%time
topk = rdd.flatMap(preprocessing) \
    .reduceByKey(add) \
    .map(token_to_key) \
    .groupByKey() \
    .flatMap(token_sum) \
    .groupByKey() \
    .map(chi_squared) \
    .sortByKey()

CPU times: user 33.1 ms, sys: 13.8 ms, total: 46.9 ms
Wall time: 10.4 s


In [ ]:
%%time
with open('../output_rdd.txt', 'w') as f:
    tokens = set()

    for category, values in topk.toLocalIterator():
        tokens.update(map(lambda x: x[1], values))
        value_strings = [f'{value[1]}:{value[0]}' for value in values]
        print(' '.join([f'<{category}>'] + value_strings), file=f)

    print(' '.join(sorted(tokens)), file=f)

CPU times: user 11.2 ms, sys: 1.62 ms, total: 12.8 ms
Wall time: 740 ms


## Part 2) Datasets/DataFrames: Spark ML and Pipelines

Convert the review texts to a classic vector space representation with TFIDF-weighted features based on the Spark DataFrame/Dataset API by building a transformation [pipeline](https://spark.apache.org/docs/latest/ml-pipeline.html). The primary goal of this part is the preparation of the pipeline for Part 3 (see below). Note: although parts of this pipeline will be very similar to Assignment 1 or Part 1 above, do not expect to obtain identical results or have access to all intermediate outputs to compare the individual steps.

Use built-in functions for [tokenization](https://spark.apache.org/docs/latest/ml-features.html#tokenizer) to unigrams at whitespaces, tabs, digits, and the delimiter characters ()\[\]{}.!?,;:+=-\_"'\`~#@&\*%€$§\\/, casefolding, [stopword removal](https://spark.apache.org/docs/latest/ml-features.html#stopwordsremover), [TF-IDF calculation](https://spark.apache.org/docs/latest/ml-features.html#tf-idf), and [chi square selection](https://spark.apache.org/docs/latest/ml-features.html#chisqselector) ) (using 2000 top terms overall). Write the terms selected this way to a file `output_ds.txt` and compare them with the terms selected in Assignment 1. Describe your observations briefly in the submission report (see Part 3).

In [4]:
df = spark.read.json(FILE_PATH)

In [5]:
tokenizer = RegexTokenizer(inputCol='reviewText', outputCol='rawTokens', pattern=r'[^a-zA-Z<>^|]+', toLowercase=True, minTokenLength=2)

In [6]:
stopwordRemover = StopWordsRemover(inputCol='rawTokens', outputCol='tokens', stopWords=list(stopwords))

In [7]:
# hashingTF = HashingTF(inputCol="tokens", outputCol="rawFeatures")
countVectorizer = CountVectorizer(inputCol="tokens", outputCol="rawFeatures")
idf = IDF(inputCol="rawFeatures", outputCol="features")

In [8]:
selector = ChiSqSelector(numTopFeatures=2000, featuresCol="features", outputCol="selectedFeatures", labelCol="overall")

In [9]:
pipeline = Pipeline(stages=[
    tokenizer,
    stopwordRemover,
    # hashingTF,
    countVectorizer,
    idf,
    selector,
])

In [10]:
model = pipeline.fit(df)

In [11]:
vocabulary = model.stages[2].vocabulary
selectedFeatures = model.stages[-1].selectedFeatures

In [12]:
chi_squared = model.transform(df).select('category', 'overall', 'selectedFeatures')

In [14]:
sparse_vector_to_map = F.udf(lambda v: {i: float(v[i]) for i in v.indices.tolist()}, returnType=MapType(IntegerType(), DoubleType()))
index_to_token = F.udf(lambda i: vocabulary[selectedFeatures[i]], returnType='string')

topk = chi_squared.select('category', F.explode(sparse_vector_to_map('selectedFeatures')).alias('token', 'chi_squared'))
topk = topk.groupBy('category', 'token').agg(F.mean('chi_squared').alias('chi_squared'))
topk = topk.withColumn('token', index_to_token('token'))
topk = topk.withColumn('rank', F.row_number().over(Window.partitionBy('category').orderBy(F.desc('chi_squared'), F.asc('token'))))
topk = topk.filter(F.col('rank') <= K)
topk = topk.withColumn('token_chisq', F.array('token', 'chi_squared'))
topk = topk.groupBy('category').agg(F.collect_list('token_chisq').alias('topk'))
topk = topk.sort('category')

In [15]:
with open('../output_ds.txt', 'w') as f:
    tokens = set()

    for row in topk.toLocalIterator():
        tokens.update(map(lambda x: x[0], row['topk']))
        value_strings = [f'{value[0]}:{value[1]}' for value in row['topk']]
        print(' '.join([f'<{row["category"]}>'] + value_strings), file=f)

    print(' '.join(sorted(tokens)), file=f)

## Part 3) Text Classification

In this part, you will train a text classifier from the features extracted in Part 2. The goal is to learn a model that can predict the product category from a review's text.

To this end, extend the pipeline from Part 2 such that a **Support Vector Machine** classifier is trained. Since we are dealing with multi-class problems, make sure to put a strategy in place that allows binary classifiers to be applicable. Apply vector length normalization before feeding the feature vectors into the classifier (use [`Normalizer`](https://spark.apache.org/docs/latest/mllib-feature-extraction.html#normalizer) with L2 norm).

Follow best practices for machine learning experiment design and investigate the effects of parameter settings using the functions provided by Spark:

*   Split the review data into training, validation, and test set.
    
*   Make experiments reproducible.
    
*   Use a grid search for parameter optimization:
    
    *   Compare chi square overall top 2000 filtered features with another, heavier filtering with much less dimensionality (see Spark ML documentation for options).
        
    *   Compare different SVM settings by varying the regularization parameter (choose 3 different values), standardization of training features (2 values), and maximum number of iterations (2 values).
        
*   Use the [`MulticlassClassificationEvaluator`](https://spark.apache.org/docs/3.5.1/ml-tuning.html#model-selection-aka-hyperparameter-tuning) to estimate performance of your trained classifiers on the test set, using F1 measure as criterion.